In [1]:
import warnings
from numpy import nan, inf, exp, log
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Metatrader_WINN_M15 (After Covid).csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M15'])
Query.head()

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]          = Query[['a','z']]
    Src[['Symbol','TF']]    = Query[['symbol','tf']]

    Src['Datetime']         = Query['datetime']
    Src['Date']             = Query['datetime'].dt.date
    Src['Time']             = Query['datetime'].dt.time
    Src['Time%']            = Query['datetime'].dt.time.apply(lambda x: x.hour + x.minute/60)

    Src_by2                 = Src.groupby(['Symbol','TF'],        sort=0, group_keys=0)
    Src_by3                 = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Src['D']                = (+1) * Src_by2['Date'].rank(method='dense')
    Src['T']                = (-1) * Src_by3['Time'].rank(method='dense', ascending=False)
    
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

# Data

In [ ]:
def CALCULATIONS(Src):
    # ================ Helpers ================ #
    def _over(Df, Ref, val, Col, get, alt=nan):
        try:    return Df[Df[Ref]==val][Col].iloc[get]
        except: return alt

    # ================ Main ================ #
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)


    Calc['Day Open']        = Calc_by['Open']   .transform('first')
    Calc['Day High']        = Calc_by['High']   .transform('max')
    Calc['Day Low']         = Calc_by['Low']    .transform('min')
    Calc['Day Close']       = Calc_by['Close']  .transform('last')

    Calc['Day Change']      = (Calc['Day Close'] - Calc['Day Open'])
    Calc['Day Variat']      = (Calc['Day Close'] / Calc['Day Open']) *100-100
    Calc['Day Hilo']        = (Calc['Day High']  - Calc['Day Low'])

    Calc['Day High Rnk']    = Calc_by['High']  .rank(ascending=False)
    Calc['Day Low Rnk']     = Calc_by['Low']   .rank(ascending=True)


    Calc['Prev Open']       = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Open',  get=(-1)), axis=1)
    Calc['Prev High']       = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day High',  get=(-1)), axis=1)
    Calc['Prev Low']        = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Low',   get=(-1)), axis=1)
    Calc['Prev Close']      = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Close', get=(-1)), axis=1)

    Calc['Prev Change']     = (Calc['Prev Close'] - Calc['Prev Open'])
    Calc['Prev Variat']     = (Calc['Prev Close'] / Calc['Prev Open']) *100-100
    Calc['Prev Hilo']       = (Calc['Prev High']  - Calc['Prev Low'])


    Calc['ID High Cumax']   = Calc_by['High'] .expanding().max()      .reset_index(drop=1)
    Calc['ID Low Cumin']    = Calc_by['Low']  .expanding().min()      .reset_index(drop=1)

    Calc['Test Up']         = (Calc['High']    - Calc['Prev High']) .apply(lambda x: 0 if (x < 0) else x)
    Calc['Test Down']       = (Calc['Prev Low'] - Calc['Low'])      .apply(lambda x: 0 if (x < 0) else x)

    Calc['ID Test Up Rnk']   = Calc_by['Test Up']    .rank(method='dense').sub(1)
    Calc['ID Test Down Rnk'] = Calc_by['Test Down']  .rank(method='dense').sub(1)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
# Calc.to_csv('Cache/Calc.csv', index=False)
# Calc = pd.read_csv('Cache/Calc.csv')
# Calc.head()

In [3]:
BINS_DIF = [500,700,1000,1300,1600,1900,2200,2500,2800,3100,3500,4000,4500,5000,inf]
BINS_VAR = [-inf, -5.00, -3.00, -2.00, -1.50, -1.00, -0.75, -0.50, -0.25, 0, +0.25, +0.50, +0.75, +1.00, +1.50, +2.00, +3.00, +5.00, +inf]

In [ ]:
def SIGNALS(Calc):
    # ================ Helpers ================ #
    def _int(x):
        try:    return int(x)
        except: return x

    # ================ Main ================ #
    Sign = pd.DataFrame(Calc)
    Sign['True']  = True
    Sign['False'] = False


    Sign['Day Is Bull']  =  Sign['Day Change'].apply(lambda x: x >= 0)
    Sign['Day Is Bear']  =  Sign['Day Change'].apply(lambda x: x <  0)

    Sign['Prev Is Bull'] = Sign['Prev Change'].apply(lambda x: x >= 0)
    Sign['Prev Is Bear'] = Sign['Prev Change'].apply(lambda x: x <  0)



    for (a,b) in [win for win in pd.Series(BINS_DIF).rolling(2) if len(win) >= 2]:
        Sign[f'Prev Hilo {_int(a)}-{_int(b)}'] = (a < Sign['Prev Hilo']) & (Sign['Prev Hilo'] < b)


    for (a,b) in [win for win in pd.Series(BINS_VAR).rolling(2) if len(win) >= 2]:
        Sign[f'Prev Variat {a}  –  {b}'] = (a < Sign['Prev Variat']) & (Sign['Prev Variat'] < b)



    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} Out Max'] =                                                  (             Sign['Prev High'] < Sign[OHLC])
        Sign[f'{OHLC} In Max']  =                                                  (Sign[OHLC] < Sign['Prev High']             )
        Sign[f'{OHLC} Inside']  = (             Sign['Prev Low'] < Sign[OHLC])  &  (Sign[OHLC] < Sign['Prev High']             )
        Sign[f'{OHLC} In Min']  = (             Sign['Prev Low'] < Sign[OHLC])
        Sign[f'{OHLC} Out Min'] = (Sign[OHLC] < Sign['Prev Low']             )


    for OHLC in ['Open','High','Low','Close']:
        Sign[f'Day {OHLC} Out Max'] =                                                                    (                      Sign['Prev High'] < Sign[f'Day {OHLC}'])
        Sign[f'Day {OHLC} In Max']  =                                                                    (Sign[f'Day {OHLC}'] < Sign['Prev High']                      )
        Sign[f'Day {OHLC} Inside']  = (                      Sign['Prev Low'] < Sign[f'Day {OHLC}'])  &  (Sign[f'Day {OHLC}'] < Sign['Prev High']                      )
        Sign[f'Day {OHLC} In Min']  = (                      Sign['Prev Low'] < Sign[f'Day {OHLC}'])
        Sign[f'Day {OHLC} Out Min'] = (Sign[f'Day {OHLC}'] < Sign['Prev Low']                      )

    return Sign

Sign = SIGNALS(Calc)
Sign.head()

# Stats

In [5]:
def STATS(Sign, BINS, conds, rank):
    # ================ Helpers ================ #
    def _to_clock(x): return round(x//1 + x%1 * 60/100, 2)

    # ================ Main ================ #
    pipe = []
    for b in BINS: 
        Df = Sign[Sign[b] & Sign[conds].all(1) & (Sign[rank]==1)]
        pipe.append({ 'Condition':b, **Df['Time%'].describe().apply(_to_clock).fillna('') }) 
    pass
    return pd.DataFrame(pipe)

# Testes e Romps (Dias Pos)

In [6]:
BINS = Sign.loc[:, f'Prev Hilo 500-700':f'Prev Hilo 5000-inf'].columns

Horário médio que fez máxima do dia

In [7]:
STATS(Sign, BINS, conds=['Prev Is Bull'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,3.0,11.50,2.01,10.0,10.45,11.30,12.45,14.00
1,Prev Hilo 700-1000,31.0,13.11,3.32,9.0,10.15,12.15,16.30,18.15
2,Prev Hilo 1000-1300,90.0,12.26,3.12,9.0,9.30,11.07,15.15,18.15
3,Prev Hilo 1300-1600,125.0,12.12,3.27,9.0,9.00,10.30,15.30,18.15
4,Prev Hilo 1600-1900,122.0,12.02,2.58,9.0,9.15,11.00,13.45,18.15
5,Prev Hilo 1900-2200,70.0,12.32,3.21,9.0,9.34,11.00,15.41,18.15
6,Prev Hilo 2200-2500,67.0,12.42,3.19,9.0,9.30,12.00,16.07,18.15
7,Prev Hilo 2500-2800,39.0,12.36,3.06,9.0,10.00,11.45,15.15,18.15
8,Prev Hilo 2800-3100,21.0,12.31,3.19,9.0,10.00,11.45,14.45,18.15
9,Prev Hilo 3100-3500,25.0,13.05,2.60,9.0,11.00,12.00,15.45,18.15


Horário Médio que fez mínima do dia

In [8]:
STATS(Sign, BINS, conds=['Prev Is Bull'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,3.0,14.55,3.59,10.30,13.15,16.00,17.07,18.15
1,Prev Hilo 700-1000,32.0,11.58,2.56,9.00,9.00,10.45,14.11,18.15
2,Prev Hilo 1000-1300,94.0,12.29,3.16,9.00,9.30,11.00,15.45,18.15
3,Prev Hilo 1300-1600,121.0,12.45,3.02,9.00,10.15,12.00,15.00,18.15
4,Prev Hilo 1600-1900,123.0,12.45,3.06,9.00,10.00,12.00,15.22,18.15
5,Prev Hilo 1900-2200,72.0,13.07,3.03,9.00,10.53,12.15,15.19,18.15
6,Prev Hilo 2200-2500,66.0,12.08,2.47,9.00,10.00,11.22,13.38,18.15
7,Prev Hilo 2500-2800,40.0,12.07,3.04,9.00,9.00,10.45,14.22,18.15
8,Prev Hilo 2800-3100,18.0,12.25,3.10,9.00,9.07,12.15,14.53,18.15
9,Prev Hilo 3100-3500,25.0,12.59,3.38,9.00,9.15,12.15,17.00,18.15


Testou a mínima no horário médio de

In [9]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close In Min'], rank='ID Test Down Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,0.0,,,,,,,
1,Prev Hilo 700-1000,8.0,12.09,2.58,9.0,9.53,11.15,14.11,16.3
2,Prev Hilo 1000-1300,24.0,13.59,3.08,9.0,11.07,14.38,16.38,18.0
3,Prev Hilo 1300-1600,35.0,13.43,2.56,9.0,11.07,14.0,16.45,17.3
4,Prev Hilo 1600-1900,28.0,13.18,3.0,9.0,10.3,13.15,15.3,18.15
5,Prev Hilo 1900-2200,7.0,11.09,2.01,9.0,9.45,10.3,12.07,14.45
6,Prev Hilo 2200-2500,4.0,14.0,4.04,9.15,11.19,14.3,17.11,17.45
7,Prev Hilo 2500-2800,2.0,12.53,3.43,10.15,11.34,12.53,14.11,15.3
8,Prev Hilo 2800-3100,4.0,14.49,2.29,12.0,13.19,14.45,16.15,17.45
9,Prev Hilo 3100-3500,0.0,,,,,,,


Testou a máxima no horário médio de:

In [10]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close In Max'], rank='ID Test Up Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,13.30,,13.30,13.30,13.30,13.30,13.30
1,Prev Hilo 700-1000,12.0,12.24,3.25,9.00,10.00,10.15,15.19,18.15
2,Prev Hilo 1000-1300,36.0,11.14,2.18,9.00,9.30,10.22,11.53,17.30
3,Prev Hilo 1300-1600,34.0,11.57,2.53,9.00,9.45,10.30,14.45,18.15
4,Prev Hilo 1600-1900,48.0,12.28,2.4,9.00,10.38,11.45,14.00,18.00
5,Prev Hilo 1900-2200,22.0,12.46,2.6,9.15,10.30,11.30,15.41,18.00
6,Prev Hilo 2200-2500,20.0,13.13,2.42,9.00,10.45,13.22,15.15,17.45
7,Prev Hilo 2500-2800,8.0,12.53,2.57,9.15,11.04,12.30,14.22,17.00
8,Prev Hilo 2800-3100,6.0,12.30,2.44,9.30,9.56,13.00,14.45,15.15
9,Prev Hilo 3100-3500,3.0,15.15,2.03,13.30,14.07,14.45,16.07,17.30


Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [11]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,10.0,,10.0,10.0,10.0,10.0,10.0
1,Prev Hilo 700-1000,3.0,10.0,0.52,9.0,9.45,10.3,10.3,10.3
2,Prev Hilo 1000-1300,11.0,9.5,0.41,9.0,9.15,9.45,10.22,10.45
3,Prev Hilo 1300-1600,5.0,9.15,0.34,9.0,9.0,9.0,9.0,10.15
4,Prev Hilo 1600-1900,5.0,10.21,1.49,9.0,9.0,9.15,11.3,13.0
5,Prev Hilo 1900-2200,2.0,9.0,0.0,9.0,9.0,9.0,9.0,9.0
6,Prev Hilo 2200-2500,0.0,,,,,,,
7,Prev Hilo 2500-2800,0.0,,,,,,,
8,Prev Hilo 2800-3100,0.0,,,,,,,
9,Prev Hilo 3100-3500,0.0,,,,,,,


Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [12]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,3.0,14.55,3.59,10.3,13.15,16.0,17.07,18.15
1,Prev Hilo 700-1000,10.0,15.13,2.19,10.3,14.11,15.3,16.3,18.15
2,Prev Hilo 1000-1300,22.0,16.11,2.15,10.0,15.45,16.38,17.45,18.15
3,Prev Hilo 1300-1600,32.0,14.44,2.31,10.0,12.11,14.45,17.0,18.15
4,Prev Hilo 1600-1900,19.0,15.56,2.1,11.3,14.3,16.3,17.52,18.15
5,Prev Hilo 1900-2200,13.0,17.01,1.1,15.0,16.3,17.3,17.45,18.15
6,Prev Hilo 2200-2500,9.0,13.47,2.31,11.15,12.0,12.3,16.0,17.45
7,Prev Hilo 2500-2800,3.0,16.0,0.26,15.45,15.45,15.45,16.07,16.3
8,Prev Hilo 2800-3100,2.0,13.22,2.18,11.45,12.34,13.22,14.11,15.0
9,Prev Hilo 3100-3500,3.0,17.15,1.19,15.45,16.45,17.45,18.0,18.15


Fechou acima da máxima de ontem e fez mínima na hora média de:

In [13]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,0.0,,,,,,,
1,Prev Hilo 700-1000,3.0,10.0,1.44,9.0,9.0,9.0,10.3,12.0
2,Prev Hilo 1000-1300,6.0,9.58,0.51,9.0,9.3,9.38,10.41,11.0
3,Prev Hilo 1300-1600,3.0,10.4,0.46,10.0,10.15,10.3,11.0,11.3
4,Prev Hilo 1600-1900,1.0,10.3,,10.3,10.3,10.3,10.3,10.3
5,Prev Hilo 1900-2200,0.0,,,,,,,
6,Prev Hilo 2200-2500,0.0,,,,,,,
7,Prev Hilo 2500-2800,1.0,9.0,,9.0,9.0,9.0,9.0,9.0
8,Prev Hilo 2800-3100,0.0,,,,,,,
9,Prev Hilo 3100-3500,0.0,,,,,,,


Fechou acima da máxima de ontem e fez máxima na hora média de:

In [14]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close Out Max'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,0.0,,,,,,,
1,Prev Hilo 700-1000,14.0,16.02,2.24,10.3,15.15,16.3,18.0,18.15
2,Prev Hilo 1000-1300,36.0,14.34,3.07,9.15,11.45,15.22,17.34,18.15
3,Prev Hilo 1300-1600,39.0,15.17,3.09,9.0,12.38,16.45,17.45,18.15
4,Prev Hilo 1600-1900,30.0,14.37,2.36,10.0,12.45,14.3,17.23,18.15
5,Prev Hilo 1900-2200,23.0,14.24,3.3,9.15,10.45,14.0,17.45,18.15
6,Prev Hilo 2200-2500,23.0,15.51,2.17,9.3,15.3,16.45,17.23,18.15
7,Prev Hilo 2500-2800,18.0,13.46,2.46,9.0,11.45,13.0,15.3,18.15
8,Prev Hilo 2800-3100,8.0,15.36,3.02,10.3,12.56,17.15,17.49,18.15
9,Prev Hilo 3100-3500,12.0,14.45,2.54,10.45,11.53,15.15,17.23,18.15


# Testes e Romps (Dias Neg)

Horário Médio que fez máxima do dia

In [15]:
STATS(Sign, BINS, conds=['Prev Is Bear'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,4.0,14.19,3.57,9.00,12.45,15.00,16.34,18.15
1,Prev Hilo 700-1000,19.0,13.30,3.33,9.00,10.15,12.30,16.45,18.15
2,Prev Hilo 1000-1300,71.0,12.05,2.57,9.00,9.30,10.45,14.22,18.15
3,Prev Hilo 1300-1600,98.0,12.07,3.02,9.00,9.30,11.00,14.30,18.15
4,Prev Hilo 1600-1900,105.0,12.49,3.05,9.00,10.00,12.15,15.30,18.15
5,Prev Hilo 1900-2200,81.0,12.53,3.05,9.00,10.15,12.00,15.45,18.15
6,Prev Hilo 2200-2500,57.0,13.04,3.11,9.00,10.15,12.45,15.45,18.15
7,Prev Hilo 2500-2800,37.0,12.28,3.01,9.00,10.00,11.45,14.00,18.15
8,Prev Hilo 2800-3100,35.0,12.51,3.05,9.00,10.30,11.45,16.00,18.15
9,Prev Hilo 3100-3500,38.0,12.15,2.46,9.00,10.00,11.45,14.30,18.15


Horário Médio que fez mínima do dia

In [16]:
STATS(Sign, BINS, conds=['Prev Is Bear'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,4.0,10.53,1.30,9.3,10.15,10.30,11.07,13.00
1,Prev Hilo 700-1000,19.0,11.23,2.20,9.0,9.53,10.45,12.15,18.15
2,Prev Hilo 1000-1300,72.0,12.28,2.54,9.0,10.26,11.30,15.15,18.15
3,Prev Hilo 1300-1600,100.0,12.44,3.13,9.0,9.45,12.00,16.15,18.15
4,Prev Hilo 1600-1900,105.0,11.39,2.51,9.0,9.00,10.30,13.00,18.15
5,Prev Hilo 1900-2200,81.0,12.41,3.18,9.0,9.30,11.30,15.45,18.15
6,Prev Hilo 2200-2500,60.0,11.47,2.30,9.0,9.30,11.00,13.38,18.15
7,Prev Hilo 2500-2800,39.0,13.21,3.18,9.0,10.30,13.00,16.38,18.15
8,Prev Hilo 2800-3100,34.0,11.51,2.50,9.0,9.15,11.00,14.00,17.45
9,Prev Hilo 3100-3500,37.0,12.33,2.59,9.0,10.15,11.15,15.15,18.15


Testou a mínima no horário médio de:

In [17]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close In Min'], rank='ID Test Down Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,2.0,10.53,2.39,9.0,9.56,10.53,11.49,12.45
1,Prev Hilo 700-1000,12.0,11.54,2.44,9.0,9.41,11.07,13.41,16.3
2,Prev Hilo 1000-1300,33.0,13.25,2.12,9.3,11.45,13.45,15.3,17.3
3,Prev Hilo 1300-1600,25.0,11.5,2.28,9.0,10.15,11.3,12.3,16.45
4,Prev Hilo 1600-1900,29.0,11.47,2.07,9.0,10.0,11.15,13.0,16.0
5,Prev Hilo 1900-2200,18.0,12.52,2.37,9.0,11.0,12.53,14.53,17.45
6,Prev Hilo 2200-2500,24.0,12.19,2.27,9.0,10.3,11.45,13.34,18.0
7,Prev Hilo 2500-2800,9.0,11.23,2.37,9.0,9.0,10.45,13.15,16.15
8,Prev Hilo 2800-3100,13.0,13.57,3.02,9.3,11.15,13.45,16.3,17.45
9,Prev Hilo 3100-3500,13.0,13.02,2.35,9.0,11.45,12.3,15.0,17.3


Testou a máxima no horário médio de:

In [18]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close In Max'], rank='ID Test Up Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,9.0,,9.0,9.0,9.0,9.0,9.0
1,Prev Hilo 700-1000,4.0,13.19,3.12,9.0,11.49,14.15,15.45,15.45
2,Prev Hilo 1000-1300,17.0,11.22,2.4,9.0,9.3,10.3,11.3,18.0
3,Prev Hilo 1300-1600,18.0,12.12,2.41,9.0,10.04,11.15,14.11,18.15
4,Prev Hilo 1600-1900,20.0,14.12,2.37,9.0,12.3,14.38,16.04,18.15
5,Prev Hilo 1900-2200,9.0,13.4,2.43,10.15,11.15,13.45,15.45,17.45
6,Prev Hilo 2200-2500,6.0,15.03,2.01,12.15,13.34,15.3,16.41,17.0
7,Prev Hilo 2500-2800,7.0,12.24,2.17,10.15,11.0,12.0,12.45,17.0
8,Prev Hilo 2800-3100,3.0,15.05,0.09,15.0,15.0,15.0,15.07,15.15
9,Prev Hilo 3100-3500,2.0,15.22,0.11,15.15,15.19,15.22,15.26,15.3


Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [19]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,9.0,,9.0,9.0,9.0,9.0,9.0
1,Prev Hilo 700-1000,1.0,15.45,,15.45,15.45,15.45,15.45,15.45
2,Prev Hilo 1000-1300,4.0,9.38,0.36,9.0,9.11,9.38,10.04,10.15
3,Prev Hilo 1300-1600,5.0,9.42,0.34,9.0,9.3,9.3,10.0,10.3
4,Prev Hilo 1600-1900,1.0,9.3,,9.3,9.3,9.3,9.3,9.3
5,Prev Hilo 1900-2200,2.0,10.45,0.21,10.3,10.38,10.45,10.53,11.0
6,Prev Hilo 2200-2500,0.0,,,,,,,
7,Prev Hilo 2500-2800,0.0,,,,,,,
8,Prev Hilo 2800-3100,0.0,,,,,,,
9,Prev Hilo 3100-3500,0.0,,,,,,,


Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [20]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,13.0,,13.0,13.0,13.0,13.0,13.0
1,Prev Hilo 700-1000,4.0,13.49,3.03,11.3,12.04,12.45,14.3,18.15
2,Prev Hilo 1000-1300,24.0,14.07,2.52,9.0,12.07,14.22,16.19,18.15
3,Prev Hilo 1300-1600,37.0,15.38,2.3,9.0,14.15,16.45,17.45,18.15
4,Prev Hilo 1600-1900,23.0,15.21,2.39,10.45,13.15,16.3,17.3,18.15
5,Prev Hilo 1900-2200,30.0,15.04,3.02,9.0,14.07,16.23,17.15,18.15
6,Prev Hilo 2200-2500,12.0,13.04,2.1,10.15,10.53,13.22,14.49,16.0
7,Prev Hilo 2500-2800,18.0,14.58,3.16,9.0,11.41,16.3,17.45,18.15
8,Prev Hilo 2800-3100,8.0,14.0,2.49,10.0,12.22,14.07,15.45,17.45
9,Prev Hilo 3100-3500,11.0,14.46,2.27,11.0,12.53,15.15,16.38,17.45


Fechou acima da máxima de ontem e fez mínima na hora média de:

In [21]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,1.0,10.3,,10.3,10.3,10.3,10.3,10.3
1,Prev Hilo 700-1000,7.0,9.47,0.52,9.0,9.0,9.3,10.3,11.0
2,Prev Hilo 1000-1300,5.0,11.06,1.2,9.0,10.45,11.3,11.45,12.3
3,Prev Hilo 1300-1600,3.0,9.05,0.09,9.0,9.0,9.0,9.07,9.15
4,Prev Hilo 1600-1900,4.0,9.38,1.15,9.0,9.0,9.0,9.38,11.3
5,Prev Hilo 1900-2200,0.0,,,,,,,
6,Prev Hilo 2200-2500,0.0,,,,,,,
7,Prev Hilo 2500-2800,0.0,,,,,,,
8,Prev Hilo 2800-3100,0.0,,,,,,,
9,Prev Hilo 3100-3500,0.0,,,,,,,


Fechou acima da máxima de ontem e fez máxima na hora média de:

In [22]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close Out Max'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Hilo 500-700,2.0,17.07,1.35,16.0,16.34,17.07,17.41,18.15
1,Prev Hilo 700-1000,9.0,16.1,2.46,10.3,16.45,16.45,18.0,18.15
2,Prev Hilo 1000-1300,14.0,15.29,2.26,10.0,14.15,15.45,17.34,18.15
3,Prev Hilo 1300-1600,24.0,14.52,2.42,9.0,13.22,15.15,16.52,18.15
4,Prev Hilo 1600-1900,29.0,15.32,2.47,9.0,14.3,16.3,17.45,18.15
5,Prev Hilo 1900-2200,18.0,16.03,1.39,11.45,15.15,16.15,17.19,18.15
6,Prev Hilo 2200-2500,10.0,16.23,1.3,13.45,15.45,16.15,17.45,18.15
7,Prev Hilo 2500-2800,2.0,13.22,0.53,12.45,13.04,13.22,13.41,14.0
8,Prev Hilo 2800-3100,3.0,18.0,0.26,17.3,17.52,18.15,18.15,18.15
9,Prev Hilo 3100-3500,1.0,16.3,,16.3,16.3,16.3,16.3,16.3


# Testes e Romps (Variat)

In [23]:
BINS = Sign.loc[:, f'Prev Variat -inf  –  -5.0':f'Prev Variat 5.0  –  inf'].columns[::-1]

Horário Médio que fez máxima do dia

In [24]:
STATS(Sign, BINS, conds=['True'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,8.0,12.58,3.38,9.0,9.22,12.53,16.38,17.0
2,Prev Variat 2.0 – 3.0,33.0,12.33,3.09,9.0,9.45,11.45,15.3,18.15
3,Prev Variat 1.5 – 2.0,64.0,12.49,3.26,9.0,9.3,11.45,16.45,18.15
4,Prev Variat 1.0 – 1.5,98.0,12.34,3.19,9.0,9.34,11.22,15.3,18.15
5,Prev Variat 0.75 – 1.0,79.0,12.28,3.17,9.0,9.3,11.15,15.22,18.15
6,Prev Variat 0.5 – 0.75,107.0,11.46,3.02,9.0,9.07,10.45,13.45,18.15
7,Prev Variat 0.25 – 0.5,117.0,12.39,3.21,9.0,9.3,11.45,16.15,18.15
8,Prev Variat 0.0 – 0.25,120.0,12.25,3.09,9.0,9.3,11.22,15.15,18.15
9,Prev Variat -0.25 – 0.0,99.0,12.38,3.17,9.0,9.3,11.3,15.3,18.15


Horário Médio que fez mínima do dia

In [25]:
STATS(Sign, BINS, conds=['True'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,7.0,12.17,3.04,9.0,10.0,11.45,13.45,17.45
2,Prev Variat 2.0 – 3.0,34.0,12.41,3.32,9.0,9.19,10.45,15.56,18.15
3,Prev Variat 1.5 – 2.0,64.0,12.22,2.49,9.0,10.11,11.45,14.34,18.15
4,Prev Variat 1.0 – 1.5,100.0,12.55,3.1,9.0,10.0,12.22,15.34,18.15
5,Prev Variat 0.75 – 1.0,79.0,12.23,2.58,9.0,10.0,11.3,14.45,18.15
6,Prev Variat 0.5 – 0.75,105.0,12.56,3.12,9.0,10.0,12.0,16.0,18.15
7,Prev Variat 0.25 – 0.5,119.0,12.42,3.14,9.0,10.0,11.45,15.45,18.15
8,Prev Variat 0.0 – 0.25,118.0,12.18,2.51,9.0,10.0,11.3,14.41,18.15
9,Prev Variat -0.25 – 0.0,104.0,12.36,3.06,9.0,10.11,11.3,15.19,18.15


Testou a mínima no horário médio de:

In [26]:
STATS(Sign, BINS, conds=['Low Out Min', 'Close In Min'], rank='ID Test Down Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,1.0,16.3,,16.3,16.3,16.3,16.3,16.3
2,Prev Variat 2.0 – 3.0,2.0,12.3,3.11,10.15,11.22,12.3,13.38,14.45
3,Prev Variat 1.5 – 2.0,7.0,14.02,2.13,11.15,12.38,13.45,15.07,17.45
4,Prev Variat 1.0 – 1.5,20.0,13.56,2.33,9.0,12.07,14.3,15.38,17.45
5,Prev Variat 0.75 – 1.0,22.0,12.21,2.21,9.15,10.38,11.53,13.22,17.45
6,Prev Variat 0.5 – 0.75,28.0,13.22,3.04,9.0,10.15,13.15,16.07,17.45
7,Prev Variat 0.25 – 0.5,37.0,13.42,3.04,9.0,10.45,13.45,16.3,18.15
8,Prev Variat 0.0 – 0.25,55.0,12.51,2.47,9.0,10.3,12.3,15.15,18.0
9,Prev Variat -0.25 – 0.0,50.0,11.51,2.02,9.0,10.19,11.15,13.15,16.3


Testou a máxima no horário médio de:

In [27]:
STATS(Sign, BINS, conds=['High Out Max', 'Close In Max'], rank='ID Test Up Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,5.0,15.21,2.4,11.45,13.15,17.0,17.15,17.3
2,Prev Variat 2.0 – 3.0,15.0,11.23,2.02,9.0,9.22,11.45,13.3,14.0
3,Prev Variat 1.5 – 2.0,34.0,13.19,2.6,9.0,10.38,13.38,15.56,18.0
4,Prev Variat 1.0 – 1.5,47.0,12.57,2.53,9.0,10.22,12.15,15.53,17.45
5,Prev Variat 0.75 – 1.0,37.0,12.02,2.28,9.0,10.0,11.3,13.15,17.3
6,Prev Variat 0.5 – 0.75,42.0,12.23,2.5,9.0,10.0,11.38,14.22,18.15
7,Prev Variat 0.25 – 0.5,55.0,12.3,2.49,9.0,10.15,11.45,14.53,18.15
8,Prev Variat 0.0 – 0.25,39.0,12.25,2.29,9.0,10.07,12.15,14.38,17.3
9,Prev Variat -0.25 – 0.0,33.0,11.51,2.17,9.0,10.3,11.15,12.45,17.3


Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [28]:
STATS(Sign, BINS, conds=['High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,0.0,,,,,,,
2,Prev Variat 2.0 – 3.0,0.0,,,,,,,
3,Prev Variat 1.5 – 2.0,1.0,9.0,,9.0,9.0,9.0,9.0,9.0
4,Prev Variat 1.0 – 1.5,2.0,9.0,0.0,9.0,9.0,9.0,9.0,9.0
5,Prev Variat 0.75 – 1.0,0.0,,,,,,,
6,Prev Variat 0.5 – 0.75,8.0,10.06,1.21,9.0,9.11,9.45,10.22,13.0
7,Prev Variat 0.25 – 0.5,6.0,9.45,1.02,9.0,9.0,9.15,10.15,11.3
8,Prev Variat 0.0 – 0.25,9.0,9.45,0.43,9.0,9.0,9.45,10.3,10.45
9,Prev Variat -0.25 – 0.0,3.0,9.4,1.09,9.0,9.0,9.0,10.0,11.0


Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [29]:
STATS(Sign, BINS, conds=['Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,1.0,14.15,,14.15,14.15,14.15,14.15,14.15
2,Prev Variat 2.0 – 3.0,1.0,15.45,,15.45,15.45,15.45,15.45,15.45
3,Prev Variat 1.5 – 2.0,5.0,15.06,2.46,12.15,12.3,15.0,17.3,18.15
4,Prev Variat 1.0 – 1.5,17.0,16.34,1.46,11.45,15.45,16.45,18.0,18.15
5,Prev Variat 0.75 – 1.0,10.0,15.36,2.05,11.15,14.19,16.07,17.0,17.45
6,Prev Variat 0.5 – 0.75,21.0,15.46,2.27,10.45,14.0,16.15,17.45,18.15
7,Prev Variat 0.25 – 0.5,25.0,16.08,2.25,10.0,15.0,17.15,17.45,18.15
8,Prev Variat 0.0 – 0.25,40.0,14.3,2.23,10.0,12.11,15.0,16.07,18.15
9,Prev Variat -0.25 – 0.0,31.0,15.36,2.29,10.3,14.0,16.45,17.23,18.15


Fechou acima da máxima de ontem e fez mínima na hora média de:

In [30]:
STATS(Sign, BINS, conds=['Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,0.0,,,,,,,
2,Prev Variat 2.0 – 3.0,1.0,9.0,,9.0,9.0,9.0,9.0,9.0
3,Prev Variat 1.5 – 2.0,0.0,,,,,,,
4,Prev Variat 1.0 – 1.5,0.0,,,,,,,
5,Prev Variat 0.75 – 1.0,1.0,11.3,,11.3,11.3,11.3,11.3,11.3
6,Prev Variat 0.5 – 0.75,2.0,10.15,0.21,10.0,10.07,10.15,10.22,10.3
7,Prev Variat 0.25 – 0.5,4.0,9.53,1.02,9.0,9.0,9.45,10.38,11.0
8,Prev Variat 0.0 – 0.25,6.0,10.07,1.08,9.0,9.3,9.38,10.41,12.0
9,Prev Variat -0.25 – 0.0,9.0,10.43,1.08,9.0,10.3,10.3,11.3,12.3


Fechou acima da máxima de ontem e fez máxima na hora média de:

In [31]:
STATS(Sign, BINS, conds=['High Out Max', 'Day Close Out Max'], rank='Day High Rnk')

,Condition,count,mean,std,min,25%,50%,75%,max
0,Prev Variat 5.0 – inf,0.0,,,,,,,
1,Prev Variat 3.0 – 5.0,3.0,14.15,2.45,11.3,12.53,14.15,15.38,17.0
2,Prev Variat 2.0 – 3.0,19.0,14.22,2.48,10.0,12.15,15.15,16.3,18.15
3,Prev Variat 1.5 – 2.0,24.0,14.52,3.17,9.3,10.56,16.45,17.45,18.15
4,Prev Variat 1.0 – 1.5,32.0,15.02,3.05,9.0,12.34,15.45,17.45,18.15
5,Prev Variat 0.75 – 1.0,27.0,15.02,2.39,9.15,13.07,15.3,17.38,18.15
6,Prev Variat 0.5 – 0.75,36.0,14.16,3.05,9.0,11.3,14.38,17.04,18.15
7,Prev Variat 0.25 – 0.5,40.0,15.09,2.52,9.0,13.04,16.23,17.45,18.15
8,Prev Variat 0.0 – 0.25,33.0,15.2,2.52,9.3,12.45,16.3,17.45,18.15
9,Prev Variat -0.25 – 0.0,34.0,15.34,2.3,9.3,14.19,16.07,17.45,18.15
